#This notebook is for the project of the course "Derivatives and Structured Products" and will answer questions from the following 4 topics:
#1. Black-Scholes-Merton Hedging
#2. The impact of the moneyness
#3. Inflation Markets
#4. CDS Valuation and Risk.

#1. Black-Scholes-Merton Hedging

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as si
import scipy.optimize as opt
import yfinance as yf
import datetime as dt

In [2]:
def DeltaHedge(S0, K, T, r, μ, σ, option_type='call', hedge_interval=1/252):
    """
    Perform Delta hedging on a European option using the Black-Scholes-Merton model.
    
    Parameters:
    S0 : float : Initial stock price
    K : float : Strike price of the option
    T : float : Time to maturity in years
    r : float : Risk-free interest rate
    μ : float : Stock price drift
    σ : float : Volatility of the underlying asset
    option_type : str : 'call' or 'put'
    hedge_interval : float : Time interval for re-hedging in years (default is daily)
    
    Returns:
    dict : A dictionary containing the hedging performance metrics
    """
    # Number of hedging steps
    N = int(T / hedge_interval)
    
    # Time points
    time_points = np.linspace(0, T, N + 1)
    
    # Simulate stock price path using Geometric Brownian Motion
    dt = hedge_interval
    S = np.zeros(N + 1)
    S[0] = S0
    for t in range(1, N + 1):
        Z = np.random.normal()
        S[t] = S[t - 1] * np.exp((μ - 0.5 * σ**2) * dt + σ * np.sqrt(dt) * Z)
    
    # Initialize variables for hedging
    cash_account = 0
    option_value = 0
    delta_previous = 0
    
    for t in range(N):
        time_to_maturity = T - time_points[t]
        
        # Calculate option price and delta
        d1 = (np.log(S[t] / K) + (r + 0.5 * σ**2) * time_to_maturity) / (σ * np.sqrt(time_to_maturity))
        d2 = d1 - σ * np.sqrt(time_to_maturity)
        
        if option_type == 'call':
            option_price = S[t] * si.norm.cdf(d1) - K * np.exp(-r * time_to_maturity) * si.norm.cdf(d2)
            delta = si.norm.cdf(d1)
        else:
            option_price = K * np.exp(-r * time_to_maturity) * si.norm.cdf(-d2) - S[t] * si.norm.cdf(-d1)
            delta = -si.norm.cdf(-d1)
        
        # Hedge adjustment
        delta_change = delta - delta_previous
        cash_account -= delta_change * S[t]
        delta_previous = delta
        
        # Update cash account with interest
        cash_account *= np.exp(r * dt)
    
    # Final option payoff
    if option_type == 'call':
        option_payoff = max(S[-1] - K, 0)
    else:
        option_payoff = max(K - S[-1], 0)

    # Total hedging portfolio value (cash + stock position)
    total_portfolio = cash_account + delta_previous * S[-1]
    
    # Replicating error: difference between portfolio and option payoff
    replicating_error = total_portfolio - option_payoff
    
    # Realized variance of log returns over the hedging period (annualized)
    log_returns = np.diff(np.log(S))
    realized_variance = np.sum(log_returns**2) / T
    
    return {
        'terminal stock price': S[-1],        # final stock price
        'option payoff': option_payoff,       # option payoff at maturity
        'realized variance': realized_variance,  # realized variance of the path
        'replicating error': replicating_error    # hedging/replication error
    }

    